# Data Cleaning

## A: Setup

In [ ]:
import os
import pandas as pd
import glob
import time
from opencage.geocoder import OpenCageGeocode
from dotenv import load_dotenv
from math import radians, sin, cos, sqrt, atan2
import requests
import math

load_dotenv()
api_key = os.getenv("geocage")

os.chdir('/Users/janlinzner/Projects/Master-Thesis-Spatial-Proximity-Venture-Capital')

## B: Companies

### Append all CSV files

In [ ]:
root_directory = 'data/companies'

df_list = []

selected_columns = [
    "Organization Name", "Organization Name URL", "Industry Groups", "Headquarters Location",
    "Founded Date", "Founded Date Precision", "Exit Date", "Exit Date Precision", "Number of Founders", "Founders",
    "Last Equity Funding Amount (in USD)", "Last Equity Funding Type",
]

for subdir, _, _ in os.walk(root_directory):
    csv_files = glob.glob(os.path.join(subdir, '*.csv'))
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

final_df = pd.concat(df_list, ignore_index=True)

companies = final_df[selected_columns].copy()

### Convert variables to date variables

In [ ]:
date_cols = ['Founded Date', 'Exit Date']
for col in date_cols:
    companies[col] = pd.to_datetime(companies[col], errors='coerce')

### Add Founding Year

In [ ]:
founded_year = companies['Founded Date'].dt.year

pos = companies.columns.get_loc('Founded Date') + 1
companies.insert(loc=pos, column='Founded Year', value=founded_year)

### Convert variables to categorical variables

In [ ]:
companies['Founded Date Precision'] = pd.Categorical(companies['Founded Date Precision'],
                                    categories=['month','year','day'],
                                    ordered=False)

companies['Exit Date Precision'] = pd.Categorical(companies['Exit Date Precision'],
                                    categories=['day','month'],
                                    ordered=False)

### Provide a company id with leading zeros

In [ ]:
companies.reset_index(drop=True, inplace=True)
companies['Company ID'] = companies.index + 1
companies['Company ID'] = companies['Company ID'].apply(lambda x: f"{x:06d}")
companies['Company ID'] = companies['Company ID'].astype('string')

cols = ['Company ID'] + [c for c in companies.columns if c != 'Company ID']
companies = companies[cols]

### Convert company events into binaries

In [ ]:
companies = companies.copy()

companies.loc[:, 'Exit Binary']     = companies['Exit Date'].notna().astype(int)
companies['Exit Binary'] = companies['Exit Binary'].astype(bool)

order = [
    'Company ID', 'Organization Name', 'Organization Name URL', 'Industry Groups', 'Headquarters Location',
    'Founded Date', 'Founded Date Precision', 'Founded Year', 'Exit Date', 'Exit Date Precision', 'Exit Binary',
    'Number of Founders', 'Founders', 'Last Equity Funding Amount (in USD)',
    'Last Equity Funding Type'
]
companies = companies[order]

### Adding geocoding

In [ ]:
cache_file = 'data/locations/location_cache.csv'

if not os.path.exists(cache_file):
    pd.DataFrame({
        'hq_location': pd.Series(dtype='object'),
        'latitude':     pd.Series(dtype='float64'),
        'longitude':    pd.Series(dtype='float64'),
    }).to_csv(cache_file, index=False)
    print(f"Created new file: {cache_file}")
else:
    print(f"File already exists: {cache_file}")

cached_coords = pd.read_csv(cache_file)

unique_locations = companies['Headquarters Location'].dropna().unique()
total = len(unique_locations)
base_url = "https://api.opencagedata.com/geocode/v1/json"
api_key   = os.getenv("geocage")

coords_list = []
for idx, location in enumerate(unique_locations, start=1):

    cached = cached_coords[cached_coords['hq_location'] == location]
    if not cached.empty:
        row = cached.iloc[0].to_dict()
        coords_list.append(row)
        print(f"[{idx}/{total}] (cached)  {location} → {row['latitude']}, {row['longitude']}")
        continue

    if not location.strip():
        row = {'hq_location': location, 'latitude': None, 'longitude': None}
        coords_list.append(row)
        print(f"[{idx}/{total}] (blank)   {location!r} → None")
    else:

        params = {"key": api_key, "q": location, "limit": 1, "no_annotations": 1}
        try:
            resp = requests.get(base_url, params=params)
            data = resp.json()
            if data['status']['code'] == 200 and data['results']:
                lat = data['results'][0]['geometry']['lat']
                lng = data['results'][0]['geometry']['lng']
            else:
                lat = lng = None
        except Exception:
            lat = lng = None

        row = {'hq_location': location, 'latitude': lat, 'longitude': lng}
        coords_list.append(row)
        print(f"[{idx}/{total}] (fetched) {location} → {lat}, {lng}")

    new_row_df = pd.DataFrame([row])
    if cached_coords.empty:
        cached_coords = new_row_df
    else:
        cached_coords = pd.concat([cached_coords, new_row_df], ignore_index=True)
    cached_coords = cached_coords.drop_duplicates(subset='hq_location', keep='first')
    cached_coords.to_csv(cache_file, index=False)

    time.sleep(0.5)

df_coords = pd.DataFrame(coords_list)
companies = companies.merge(
    df_coords,
    left_on='Headquarters Location',
    right_on='hq_location',
    how='left'
).drop(columns=['hq_location'])

lat = companies.pop('latitude')
lon = companies.pop('longitude')

insert_at = companies.columns.get_loc('Headquarters Location') + 1

companies.insert(insert_at, 'latitude', lat)
companies.insert(insert_at + 1, 'longitude', lon)

companies = (
    companies
      .rename(columns={
          'latitude':  'Latitude',
          'longitude': 'Longitude'
      })
)

### Startup Hub
https://www.startupblink.com <br>
Top 100

In [ ]:
geocoder = OpenCageGeocode(api_key)

hotspot_names = [
    "London, United Kingdom", "Paris, France", "Berlin, Germany",
    "Stockholm, Sweden", "Munich, Germany", "Helsinki, Finland",
    "Madrid, Spain", "Dublin, Ireland", "Tallinn, Estonia",
    "Copenhagen, Denmark", "Milan, Italy", "Zurich, Switzerland",
    "Oslo, Norway", "Cambridge, United Kingdom", "Kyiv, Ukraine",
    "Vienna, Austria", "Brussels, Belgium", "Manchester, United Kingdom",
    "Lisbon, Portugal", "Prague, Czech Republic", "Warsaw, Poland",
    "Hamburg, Germany", "Oxford, United Kingdom", "Amsterdam, The Netherlands", "Barcelona, Spain", "Lausanne, Switzerland",
]

records = []
for name in hotspot_names:
    results = geocoder.geocode(name, limit=5)
    if results:
        best = max(results, key=lambda x: x.get("confidence", 0))
        geom = best["geometry"]
        records.append({
            "city": name.split(",", 1)[0],
            "lat":  geom["lat"],
            "lng":  geom["lng"]
        })
    time.sleep(0.2)  # rate‐limit
hotspots_df = pd.DataFrame(records)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

def min_dist_to_hotspot(lat, lon):
    return hotspots_df.apply(
        lambda r: haversine(lat, lon, r["lat"], r["lng"]), axis=1
    ).min()

companies["Latitude"]  = pd.to_numeric(companies["Latitude"],  errors="coerce")
companies["Longitude"] = pd.to_numeric(companies["Longitude"], errors="coerce")

companies["Distance to Hub"] = companies.apply(
    lambda r: min_dist_to_hotspot(r["Latitude"], r["Longitude"]), axis=1
)
companies["Hub Binary"] = (companies["Distance to Hub"] < 20).astype(int)

In [ ]:
geocoder = OpenCageGeocode(api_key)

hotspot_names = [
    "London, United Kingdom", "Paris, France", "Berlin, Germany",
    "Stockholm, Sweden", "Munich, Germany", "Helsinki, Finland",
    "Madrid, Spain", "Dublin, Ireland", "Tallinn, Estonia",
    "Copenhagen, Denmark", "Milan, Italy", "Zurich, Switzerland",
    "Oslo, Norway", "Cambridge, United Kingdom", "Kyiv, Ukraine",
    "Vienna, Austria", "Brussels, Belgium", "Manchester, United Kingdom",
    "Lisbon, Portugal", "Prague, Czech Republic", "Warsaw, Poland",
    "Hamburg, Germany", "Oxford, United Kingdom", "Amsterdam, The Netherlands", "Barcelona, Spain", "Lausanne, Switzerland",
]

records = []
for name in hotspot_names:
    city, country = [s.strip() for s in name.split(",", 1)]
    results = geocoder.geocode(name, limit=5)
    if results:
        best = max(results, key=lambda x: x.get("confidence", 0))
        geom = best["geometry"]
        records.append({
            "city":    city,
            "country": country,
            "lat":     geom["lat"],
            "lng":     geom["lng"]
        })
    time.sleep(0.2)  # respect rate‐limit

hotspots_df = pd.DataFrame(records)

hotspots_df.to_csv("data/sets-for-r/startup_hubs.csv", index=False)

print("Wrote", len(hotspots_df), "hubs to startup_hubs.csv")

### Print final companies dataframe

In [ ]:
companies

## C: Rounds

### Append all CSV files

In [ ]:
root_directory = 'data/rounds'

selected_columns = [
    'Transaction Name', 'Transaction Name URL', 'Organization Name', 'Organization Name URL', 'Funding Type', 'Money Raised (in USD)', 'Announced Date', 'Lead Investors', 'Investor Names', 'Number of Investors',
]

df_list = []

for subdir, _, _ in os.walk(root_directory):
    csv_files = glob.glob(os.path.join(subdir, '*.csv'))
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

final_df = pd.concat(df_list, ignore_index=True)

rounds = final_df[selected_columns].copy()

### Convert variables to date variables

In [ ]:
date_cols = ['Announced Date']
for col in date_cols:
    rounds[col] = pd.to_datetime(rounds[col], errors='coerce')

### Add round id with leading zeros

In [ ]:
rounds.reset_index(drop=True, inplace=True)
rounds['Round ID'] = rounds.index + 1
rounds['Round ID'] = rounds['Round ID'].apply(lambda x: f"{x:06d}")
rounds['Round ID'] = rounds['Round ID'].astype('string')

cols = ['Round ID'] + [c for c in rounds.columns if c != 'Round ID']
rounds = rounds[cols]

## D: Investors

### Append all CSV files

In [ ]:
root_directory = 'data/investors'
df_list = []

selected_columns = [
    "Organization/Person Name", "Organization/Person Name URL", "Investor Type",
    "Number of Investments", "Number of Exits", "Location",
    "Description", "Number of Lead Investments",
    "Number of Portfolio Organizations", "Founded Date", "Industry Groups",
]

for subdir, _, _ in os.walk(root_directory):
    csv_files = glob.glob(os.path.join(subdir, '*.csv'))
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

final_df = pd.concat(df_list, ignore_index=True)

investors = final_df[selected_columns].copy()

investors.drop_duplicates(
    subset=['Organization/Person Name', 'Organization/Person Name URL'],
    inplace=True
)

### Convert variables to date variables

In [ ]:
date_cols = ['Founded Date']
for col in date_cols:
    investors[col] = pd.to_datetime(investors[col], errors='coerce')

### Add investor id with leading zeros

In [ ]:
investors.reset_index(drop=True, inplace=True)
investors['Investor ID'] = investors.index + 1
investors['Investor ID'] = investors['Investor ID'].apply(lambda x: f"{x:06d}")
investors['Investor ID'] = investors['Investor ID'].astype('string')

cols = ['Investor ID'] + [c for c in investors.columns if c != 'Investor ID']
investors = investors[cols]

### Adding Geocoding

In [ ]:
cache_file = 'data/locations/location_cache.csv'
if not os.path.exists(cache_file):
    pd.DataFrame({
        'hq_location': pd.Series(dtype='object'),
        'latitude':     pd.Series(dtype='float64'),
        'longitude':    pd.Series(dtype='float64'),
    }).to_csv(cache_file, index=False)
    print(f"Created new file: {cache_file}")
else:
    print(f"File already exists: {cache_file}")

cached_coords = pd.read_csv(cache_file)

unique_locations = investors['Location'].dropna().unique()
total = len(unique_locations)
base_url = "https://api.opencagedata.com/geocode/v1/json"
api_key   = os.getenv("geocage")

coords_list = []
for idx, location in enumerate(unique_locations, start=1):

    cached = cached_coords[cached_coords['hq_location'] == location]
    if not cached.empty:
        row = cached.iloc[0].to_dict()
        coords_list.append(row)
        print(f"[{idx}/{total}] (cached)  {location} → {row['latitude']}, {row['longitude']}")
        continue

    if not location.strip():
        row = {'hq_location': location, 'latitude': None, 'longitude': None}
        coords_list.append(row)
        print(f"[{idx}/{total}] (blank)   {location!r} → None")
    else:

        params = {"key": api_key, "q": location, "limit": 1, "no_annotations": 1}
        try:
            resp = requests.get(base_url, params=params)
            data = resp.json()
            if data['status']['code'] == 200 and data['results']:
                lat = data['results'][0]['geometry']['lat']
                lng = data['results'][0]['geometry']['lng']
            else:
                lat = lng = None
        except Exception:
            lat = lng = None

        row = {'hq_location': location, 'latitude': lat, 'longitude': lng}
        coords_list.append(row)
        print(f"[{idx}/{total}] (fetched) {location} → {lat}, {lng}")

    new_row_df = pd.DataFrame([row])
    cached_coords = pd.concat([cached_coords, new_row_df], ignore_index=True)
    cached_coords = cached_coords.drop_duplicates(subset='hq_location', keep='first')
    cached_coords.to_csv(cache_file, index=False)
    time.sleep(0.5)

df_coords = pd.DataFrame(coords_list)

investors = investors.merge(
    df_coords,
    left_on='Location',
    right_on='hq_location',
    how='left'
).drop(columns=['hq_location'])

lat = investors.pop('latitude')
lon = investors.pop('longitude')
insert_at = investors.columns.get_loc('Location') + 1
investors.insert(insert_at,     'Latitude',  lat)
investors.insert(insert_at + 1, 'Longitude', lon)

### Add zeros for NaN in Lead Investments and Exits

In [ ]:
investors['Number of Lead Investments'] = investors['Number of Lead Investments'].fillna(0)
investors['Number of Exits'] = investors['Number of Exits'].fillna(0)

In [ ]:
investors.columns

### Extract Descriptions to classify it if it is a sector specific investor

In [ ]:
focus_path = 'data/industry-focus/industry_focus_save.csv'
existing = pd.read_csv(focus_path)

merged = investors.merge(
    existing[['Organization/Person Name', 'Organization/Person Name URL', 'Specific VC Binary']],
    on=['Organization/Person Name', 'Organization/Person Name URL'],
    how='left'
)

annot_df = merged[[
    'Organization/Person Name',
    'Description',
    'Organization/Person Name URL',
    'Specific VC Binary'
]].copy()

annot_df['Specific VC Binary'] = annot_df['Specific VC Binary'] \
    .map({1: '1', 0: '0'}) \
    .fillna('')

to_annotate = annot_df[annot_df['Specific VC Binary'] == '']

to_annotate.to_csv(
    'data/industry-focus/industry_focus.csv',
    index=False
)

In [ ]:
industry_focus = pd.read_csv('data/industry-focus/industry_focus_save.csv')

investors = investors.merge(
    industry_focus[['Organization/Person Name',
                    'Organization/Person Name URL',
                    'Specific VC Binary']],
    on=['Organization/Person Name', 'Organization/Person Name URL'],
    how='left'
)

investors['Specific VC Binary'] = (
    investors['Specific VC Binary']
    .fillna(0)
    .astype(int)
    .astype('boolean')
)

### Extract Investor Type
Search the 'Investor Type' column and if there is one of the Types, put a True in the assigned Boolean.

In [ ]:
def parse_types(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return [t.strip().lower() for t in x]

    return [t.strip().lower() for t in x.split(",")]


keywords = {
    "accelerator":              "Accelerator",
    "micro vc":                 "Micro VC",
    "corporate venture capital":"Corporate Venture Capital",
    "angel group":              "Angel Group"
}

investors["type_list"] = investors["Investor Type"].apply(parse_types)

for kw, col in keywords.items():
    investors[col] = investors["type_list"].apply(lambda lst: kw in lst)

investors.drop(columns="type_list", inplace=True)

### Print final investors dataframe

In [ ]:
investors

## E: Seed Setup

### Seed - Help Table

Create Industry Group Binaries

In [ ]:
companies['Industry Groups'] = companies['Industry Groups'].fillna('').str.replace(r'\s*,\s*', ',', regex=True)
dummies = companies['Industry Groups'].str.get_dummies(sep=',').astype(bool)
companies = pd.concat([companies, dummies], axis=1)

Delete companies from the Blockchain environment due to different characteristics

In [ ]:
companies = companies[~companies['Blockchain and Cryptocurrency']]

In [ ]:
round_type_col = 'Funding Type'
seed_rounds = rounds[rounds[round_type_col].isin(['Seed', 'Pre-Seed', 'Angel'])].copy()

company_lookup = {
    (str(n).strip().lower(), str(u).strip().lower()): cid
    for n, u, cid in zip(
        companies['Organization Name'],
        companies['Organization Name URL'],
        companies['Company ID']
    )
}

investor_lookup = {
    str(n).strip().lower(): (iid, str(url).strip())
    for n, iid, url in zip(
        investors['Organization/Person Name'],
        investors['Investor ID'],
        investors['Organization/Person Name URL']
    )
}

seed_rounds['comp_key'] = seed_rounds.apply(
    lambda r: (str(r['Organization Name']).strip().lower(),
               str(r['Organization Name URL']).strip().lower()),
    axis=1
)
seed_rounds = seed_rounds[seed_rounds['comp_key'].isin(company_lookup)].copy()
seed_rounds.drop(columns=['comp_key'], inplace=True)

records = []
for _, row in seed_rounds.iterrows():
    rid = row['Round ID']
    org_name = row['Organization Name']
    org_url = row['Organization Name URL']
    comp_key = (org_name.strip().lower(), org_url.strip().lower())
    company_id = company_lookup[comp_key]
    lead_raw = row.get('Lead Investors', '')
    lead_list = [inv.strip() for inv in str(lead_raw).split(',') if pd.notna(lead_raw) and inv.strip().lower()!='nan']
    inv_raw = row.get('Investor Names', '')
    inv_list = [inv.strip() for inv in str(inv_raw).split(',') if pd.notna(inv_raw) and inv.strip().lower()!='nan']
    for inv_name in set(lead_list + inv_list):
        lookup = investor_lookup.get(inv_name.lower())
        if not lookup:
            continue
        inv_id, inv_url = lookup
        records.append({
            'Round ID':               rid,
            'Company ID':             company_id,
            'Organization Name':      org_name,
            'Organization Name URL':  org_url,
            'Investor ID':            inv_id,
            'Investor Name':          inv_name,
            'Investor URL':           inv_url,
            'Lead':                   inv_name in lead_list
        })

seed_help = pd.DataFrame(records)

investor_extra = [
    'Number of Investments',
    'Number of Portfolio Organizations',
    'Number of Lead Investments',
    'Number of Exits',
    'Longitude',
    'Latitude',
    'Founded Date',
    'Specific VC Binary',
    'Accelerator',
    'Micro VC',
    'Angel Group',
    'Corporate Venture Capital'
]
seed_help = seed_help.merge(
    investors[['Investor ID'] + investor_extra],
    on='Investor ID',
    how='left'
)

company_extra = ['Longitude', 'Latitude', 'Founded Year']
seed_help = seed_help.merge(
    companies[['Company ID'] + company_extra],
    on='Company ID',
    how='left',
    suffixes=('', '_comp')
)

rename_map = {
    'Longitude':           'Investor Longitude',
    'Latitude':            'Investor Latitude',
    'Longitude_comp':      'Company Longitude',
    'Latitude_comp':       'Company Latitude',
    'Founded Date':       'Investor Founded Date',
    'Founded Year':      'Organization Founded Year',
}
seed_help.rename(columns=rename_map, inplace=True)

cols_order = [
    'Round ID',
    'Company ID',
    'Organization Name',
    'Organization Name URL',
    'Organization Founded Year',
    'Investor ID',
    'Investor Name',
    'Investor URL',
    'Investor Founded Date',
    'Investor Longitude',
    'Investor Latitude',
    'Company Longitude',
    'Company Latitude',
    'Number of Investments',
    'Number of Portfolio Organizations',
    'Number of Lead Investments',
    'Number of Exits',
    'Lead',
    'Specific VC Binary',
    'Accelerator',
    'Angel Group',
    'Micro VC',
    'Corporate Venture Capital'
]
seed_help = seed_help[cols_order]

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

seed_help['Distance km'] = seed_help.apply(
    lambda r: haversine(
        r['Investor Latitude'], r['Investor Longitude'],
        r['Company Latitude'], r['Company Longitude']
    ), axis=1
)
seed_help['Local Investor'] = seed_help['Distance km'] < 100

### Create features to companies dataframe about seed round

#### Number of Seed Rounds

In [ ]:
counts = seed_help.groupby('Company ID')['Round ID'].nunique().rename('Number Seed Rounds')
companies = companies.merge(counts, on='Company ID', how='left')
companies['Number Seed Rounds'] = companies['Number Seed Rounds'].fillna(0).astype(int)

#### Average Distance to Seed Investors

In [ ]:
avg_distance = seed_help.groupby('Company ID')['Distance km'].mean().rename('Avg Seed Investor Distance')
companies = companies.merge(avg_distance, on='Company ID', how='left')
companies['Avg Seed Investor Distance'] = companies['Avg Seed Investor Distance'].fillna(0)

#### Average Portfolio Organizations of Investors

In [ ]:
seed_help['Number of Portfolio Organizations'] = pd.to_numeric(
    seed_help['Number of Portfolio Organizations'],
    errors='coerce'
)

avg_portfolio = (
    seed_help
    .groupby('Company ID')['Number of Portfolio Organizations']
    .mean()
    .rename('Average Seed Investors Portfolio Organizations')
)

companies = companies.merge(
    avg_portfolio,
    on='Company ID',
    how='left'
)

companies['Average Seed Investors Portfolio Organizations'] = (
    companies['Average Seed Investors Portfolio Organizations']
    .fillna(0)
)

#### Number of Seed Investors

In [ ]:
num_seed_investors = (
    seed_help
    .groupby('Company ID')['Investor ID']
    .nunique()
    .rename('Number Seed Investors')
)

companies = companies.merge(
    num_seed_investors,
    on='Company ID',
    how='left'
)

companies['Number Seed Investors'] = companies['Number Seed Investors'].fillna(0).astype(int)

#### Binary for Regional Investor and Overregional Investor

In [ ]:
regional = (
    seed_help
    .groupby('Company ID')['Local Investor']
    .any()
    .rename('Regional Seed Investor Binary')
    .fillna(False)
    .astype(bool)
)

overregional = (
    seed_help
    .groupby('Company ID')['Local Investor']
    .apply(lambda x: (~x).any())
    .rename('Overregional Seed Investor Binary')
    .fillna(False)
    .astype(bool)
)

companies = (
    companies
    .merge(regional, on='Company ID', how='left')
    .merge(overregional, on='Company ID', how='left')
)

Number of Lead Investors and Binary for Regional and Overregional Investor

In [ ]:
num_lead_seed_investors = (
    seed_help[seed_help['Lead']]
    .groupby('Company ID')['Investor ID']
    .nunique()
    .rename('Number Lead Seed Investors')
)

has_regional_lead = (
    seed_help[seed_help['Lead']]
    .groupby('Company ID')['Local Investor']
    .any()
    .rename('Regional Lead Seed Investor Binary')
    .fillna(False)
    .astype(bool)
)

has_overregional_lead = (
    seed_help[seed_help['Lead']]
    .groupby('Company ID')['Local Investor']
    .apply(lambda x: (~x).any())
    .rename('Overregional Lead Seed Investor Binary')
    .fillna(False)
    .astype(bool)
)

companies = (
    companies
    .merge(num_lead_seed_investors, on='Company ID', how='left')
    .merge(has_regional_lead, on='Company ID', how='left')
    .merge(has_overregional_lead, on='Company ID', how='left')
)

companies['Number Lead Seed Investors'] = companies['Number Lead Seed Investors'].fillna(0).astype(int)

companies['Number Lead Seed Investors'] = companies['Number Lead Seed Investors'].fillna(0).astype(int)

mask_no_lead = companies['Number Lead Seed Investors'] == 0
companies.loc[mask_no_lead, 'Regional Lead Seed Investor Binary'] = False
companies.loc[mask_no_lead, 'Overregional Lead Seed Investor Binary'] = False

#### Make the binaries to a boolean

In [ ]:
companies = companies.convert_dtypes()

binary_cols = [
    'Regional Seed Investor Binary',
    'Overregional Seed Investor Binary',
    'Regional Lead Seed Investor Binary',
    'Overregional Lead Seed Investor Binary'
]

for col in binary_cols:
    companies[col] = companies[col].fillna(False)

#### Average Age of Seed Investors

In [ ]:
if 'Announced Date' not in seed_help.columns:
    seed_help = seed_help.merge(
        rounds[['Round ID', 'Announced Date']],
        on='Round ID',
        how='left'
    )

seed_help['Announced Date']        = pd.to_datetime(seed_help['Announced Date'], errors='coerce')
seed_help['Investor Founded Date'] = pd.to_datetime(seed_help['Investor Founded Date'], errors='coerce')

seed_help['Investment Year']       = seed_help['Announced Date'].dt.year
seed_help['Investor Founded Year'] = seed_help['Investor Founded Date'].dt.year

seed_help['Investor Age at Investment'] = (
    (seed_help['Investment Year'] - seed_help['Investor Founded Year'])
    .clip(lower=0)
)

avg_age = (
    seed_help
    .groupby('Company ID')['Investor Age at Investment']
    .mean()
    .round(1)
    .rename('Avg Age of Seed Investors')
)

companies = companies.merge(
    avg_age,
    on='Company ID',
    how='left'
)
companies['Avg Age of Seed Investors'] = companies['Avg Age of Seed Investors'].fillna(0)

#### Average Number of Exits of Seed Investors

In [ ]:
avg_exits = (
    seed_help
    .groupby('Company ID')['Number of Exits']
    .mean()
    .round(1)
    .rename('Avg Exits of Seed Investors')
)

companies = companies.merge(
    avg_exits,
    on='Company ID',
    how='left'
)
companies['Avg Exits of Seed Investors'] = companies['Avg Exits of Seed Investors'].fillna(0)

#### Binary Industry Focused Investor in Seed

In [ ]:
seed_help['Specific VC Binary'] = (
    seed_help['Specific VC Binary']
    .astype('boolean')
)

industry_specific = (
    seed_help
    .groupby('Company ID')['Specific VC Binary']
    .any()
    .rename('Specific VC in Seed Binary')
)

companies = companies.merge(
    industry_specific,
    on='Company ID',
    how='left'
)

companies['Specific VC in Seed Binary'] = (
    companies['Specific VC in Seed Binary']
    .fillna(False)
    .astype('boolean')
)


#### Industry Specific VC in Seed Binary

In [ ]:
industry_specific_lead = (
    seed_help[seed_help['Lead'] & seed_help['Specific VC Binary']]
    .groupby('Company ID')['Specific VC Binary']
    .any()
    .rename('Specific Lead VC in Seed Binary')
)

companies = companies.merge(
    industry_specific_lead,
    on='Company ID',
    how='left'
)

companies['Specific Lead VC in Seed Binary'] = (
    companies['Specific Lead VC in Seed Binary']
    .fillna(False)
    .astype('boolean')
)

### Accelerator, Micro VC, Angel Group or Corporate Venture Capita in Seed

In [ ]:
flags = [
    "Accelerator",
    "Angel Group",
    "Micro VC",
    "Corporate Venture Capital",
]

company_flags = (
    seed_help
    .groupby("Company ID")[flags]
    .any()   
    .rename(columns=lambda c: f"{c} Funding Binary")
)

companies = companies.merge(
    company_flags,
    on="Company ID",
    how="left"
)

for col in company_flags.columns:
    companies[col] = companies[col].fillna(False).astype("boolean")


In [ ]:
flags = [
    "Accelerator",
    "Angel Group",
    "Micro VC",
    "Corporate Venture Capital",
]

lead_rounds = seed_help[seed_help["Lead"] == True]

company_lead_flags = (
    lead_rounds
    .groupby("Company ID")[flags]
    .any() 
    .rename(columns=lambda c: f"{c} Lead Funding Binary")
)

companies = companies.merge(
    company_lead_flags,
    on="Company ID",
    how="left"
)

for col in company_lead_flags.columns:
    companies[col] = companies[col].fillna(False).astype("boolean")

#### Time to first Seed

In [ ]:
if 'Investment Year' not in seed_help.columns:
    seed_help['Investment Year'] = pd.to_datetime(
        seed_help['Announced Date'], errors='coerce'
    ).dt.year

first_seed = (
    seed_help
    .groupby('Company ID')['Investment Year']
    .min()
    .rename('First Seed Year')
)

companies = companies.merge(
    first_seed,
    on='Company ID',
    how='left'
)

companies['Years to Seed'] = (
    (companies['First Seed Year'] - companies['Founded Year'])
    .clip(lower=0)
    .fillna(0)
    .astype(int)
)

companies.drop(columns=['First Seed Year'], inplace=True)

## F: Success

In addition to the Exit Binary, another success for an VC investor is to get the company to a certain investment stage as it minimizes the risk of losing the investment. Therefore, we define certain stages where it is assumed as an successful investment. The stages are:

- Series B
- Series C
- Series D
- Series E
- Series F
- Series G
- Private Equity with Last Equity Funding Amount (in USD) > 20,000,000
- Post-IPO Equity with Last Equity Funding Amount (in USD) > 20,000,000

Look for unique values

In [ ]:
unique_vals = companies['Last Equity Funding Type'].dropna().unique().tolist()
print(unique_vals)

Define Funding Success according to the rules set before

In [ ]:
series_success = ['Series B', 'Series C', 'Series D', 'Series E', 'Series F', 'Series G']

companies['Last Equity Funding Amount (in USD)'] = pd.to_numeric(
    companies['Last Equity Funding Amount (in USD)'],
    errors='coerce'
).fillna(0)

companies['Funding Success'] = (
    companies['Last Equity Funding Type'].isin(series_success) |
    (
        (companies['Last Equity Funding Type'] == 'Private Equity') &
        (companies['Last Equity Funding Amount (in USD)'] > 20_000_000)
    ) |
    (
        (companies['Last Equity Funding Type'] == 'Post-IPO Equity') &
        (companies['Last Equity Funding Amount (in USD)'] > 20_000_000)
    )
).astype(bool)

Define Success binary / boolean

In [ ]:
companies['Success'] = (
    companies['Exit Binary'].fillna(False) |
    companies['Funding Success'].fillna(False)
).astype(bool)

cols = [c for c in companies.columns if c not in ['Exit Binary', 'Success']]
new_order = cols + ['Exit Binary', 'Success']
companies = companies[new_order]

## G: Prepare Companies Data Frame for Analysis in R

Delete companies without founding round due to errors in Crunchbase filtering

In [ ]:
companies = companies[companies['Number Seed Rounds'] != 0]

Create country as a control

In [ ]:
companies['Headquarters Country'] = companies['Headquarters Location'].str.split(',').str[-1].str.strip()

insert_at = companies.columns.get_loc('Headquarters Location') + 1
companies.insert(insert_at, 'Headquarters Country', companies.pop('Headquarters Country'))

Only select companies from a certain year

In [ ]:
start_year = 2007
end_year = 2025

companies = companies[companies['Founded Year'].between(start_year, end_year)]

## H: Create an investor data frame
Based on the investments in the time frame and prepare it for R analysis

Aggregate features for the investors based on their investment behaviours

In [ ]:
seed_help_enriched = seed_help.merge(
    companies[['Company ID', 'Industry Groups', 'Founded Year', 'Success', 'Hub Binary', 'Number Seed Rounds', 'Headquarters Country']],
    on='Company ID',
    how='left'
)


investor_aggregates = seed_help_enriched.groupby('Investor ID').agg(
    Number_of_Seed_Investments              = ('Round ID', 'nunique'),
    Number_of_Companies_Invested            = ('Company ID', 'nunique'),
    Number_of_Unique_Industries_Invested    = ('Industry Groups',
                                               lambda x: x.str.get_dummies(sep=',').sum().gt(0).sum()),
    Average_Company_Age_at_Investment       = ('Organization Founded Year',
                                               lambda x: (seed_help_enriched['Investment Year'] - x).mean()),
    Number_of_Successful_Investments        = ('Success', 'sum'),
    Number_of_Local_Investments             = ('Local Investor', 'sum'),
    Average_Investment_Distance             = ('Distance km', 'mean'),
    Number_of_Investments_in_Hubs           = ('Hub Binary', 'sum'),
    Average_Number_of_Seed_Rounds           = ('Number Seed Rounds', 'mean'),
    Industry_Specific_Investments           = ('Specific VC Binary', 'sum'),
    Number_of_Unique_Countries_Invested     = ('Headquarters Country', 'nunique'),
    Number_of_Investments_with_Fundraising_Success = ('Success', 'sum'),
    Number_of_Lead_Seed_Investments         = ('Lead', 'sum')          
).reset_index()

rename_features = {
    'Number_of_Seed_Investments':               'Seed Investments Count',
    'Number_of_Companies_Invested':             'Unique Companies Count',
    'Number_of_Unique_Industries_Invested':     'Unique Industries Count',
    'Average_Company_Age_at_Investment':        'Avg Company Age at Investment',
    'Number_of_Successful_Investments':         'Successful Investments Count',
    'Number_of_Local_Investments':              'Local Investments Count',
    'Average_Investment_Distance':              'Avg Investment Distance (km)',
    'Number_of_Investments_in_Hubs':            'Hub Investments Count',
    'Average_Number_of_Seed_Rounds':            'Avg Seed Rounds per Company',
    'Industry_Specific_Investments':            'Specific Investments Count',
    'Number_of_Unique_Countries_Invested':      'Unique Countries Count',
    'Number_of_Investments_with_Fundraising_Success': 'Fundraising Success Count',
    'Number_of_Lead_Seed_Investments':          'Seed Lead Investments Count'  
}

investor_aggregates.rename(columns=rename_features, inplace=True)

investors_seed = investors.merge(investor_aggregates, on='Investor ID', how='left')

Select needed features

In [ ]:
numeric_columns = investors_seed.select_dtypes(include=['float64', 'int64']).columns
investors_seed[numeric_columns] = investors_seed[numeric_columns].apply(pd.to_numeric, errors='coerce')

required_columns = [
    'Investor ID', 'Organization/Person Name', 'Organization/Person Name URL', 'Description', 'Location',
    'Seed Investments Count', 'Seed Lead Investments Count', 'Unique Companies Count', 'Unique Industries Count',
    'Local Investments Count', 'Avg Investment Distance (km)', 'Hub Investments Count',
    'Unique Countries Count', 'Fundraising Success Count', 'Successful Investments Count', 'Specific VC Binary', 'Accelerator', 'Corporate Venture Capital', 'Micro VC', 'Angel Group'
]

investors_seed = investors_seed[required_columns]


Create Country variable

In [ ]:
investors_seed['Country'] = investors_seed['Location'].str.split(',').str[-1].str.strip()

insert_at = investors_seed.columns.get_loc('Location') + 1
investors_seed.insert(insert_at, 'Country', investors_seed.pop('Country'))

Create Median Distance, create the local investor variable, and implement restriction on the data quality (seed investment not NaN and Avg Investment Distance not NaN)

In [ ]:
investors_seed['Local Investor'] = investors_seed['Avg Investment Distance (km)'] < 100

investors_seed = investors_seed.dropna(subset=['Seed Investments Count'])

investors_seed = investors_seed.dropna(subset=['Avg Investment Distance (km)'])

Primary Role in Investments (based on the majority)

In [ ]:
round_counts = (
    seed_help
    .groupby("Round ID")["Investor ID"]
    .nunique()
    .rename("num_investors")
)

seed_help = seed_help.merge(round_counts, on="Round ID", how="left")

def assign_role(row):
    if row["Lead"]:
        return "Lead"
    else:
        return "Co-Investor"

seed_help["Syndication Role"] = seed_help.apply(assign_role, axis=1)

primary_roles = (
    seed_help
    .groupby("Investor ID")["Syndication Role"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
    .rename("Primary Role")
)

investors_seed = investors_seed.merge(
    primary_roles,
    on="Investor ID",
    how="left"
)

investors_seed["Lead Investor"] = investors_seed["Primary Role"] == "Lead"

investors_seed = investors_seed.drop(columns=["Primary Role"])

investors_seed["Lead Investor"] = investors_seed["Lead Investor"].astype("boolean")

Hub Investor

In [ ]:
investors_seed["Hub Investor"] = (
    investors_seed["Hub Investments Count"] /
    investors_seed["Seed Investments Count"]
) > 0.75

Only select investors with at least a certain number of investments

In [ ]:
min_seed_investments = 1
investors_seed = investors_seed[investors_seed['Seed Investments Count'] >= min_seed_investments]

Ordering of dataframe

In [ ]:
logical_order = [
    'Investor ID', 'Organization/Person Name', 'Organization/Person Name URL', 'Location', 'Country', 'Accelerator', 'Angel Group', 'Micro VC', 
    'Corporate Venture Capital', 'Specific VC Binary', 'Local Investor', 'Hub Investor', 'Lead Investor', 'Seed Investments Count', 'Seed Lead Investments Count', 
    'Unique Companies Count', 'Unique Industries Count', 'Local Investments Count', 'Avg Investment Distance (km)', 'Hub Investments Count',
    'Unique Countries Count', 'Fundraising Success Count', 'Successful Investments Count'
]
investors_seed = investors_seed[logical_order]

Final output

In [ ]:
investors_seed

## I: Save the dataframes

In [ ]:
keep_types = ["Pre-Seed", "Seed", "Angel"]
rounds_filtered = rounds[rounds["Funding Type"].isin(keep_types)].copy()
rounds_filtered.reset_index(drop=True, inplace=True)

In [ ]:
companies_seed = companies.copy()

to_move = [
    "Administrative Services", "Advertising", "Agriculture and Farming",
    "Apps", "Artificial Intelligence (AI)", "Biotechnology",
    "Blockchain and Cryptocurrency", "Clothing and Apparel",
    "Commerce and Shopping", "Community and Lifestyle",
    "Consumer Electronics", "Consumer Goods", "Content and Publishing",
    "Data and Analytics", "Design", "Education", "Energy", "Events",
    "Financial Services", "Food and Beverage", "Gaming",
    "Government and Military", "Hardware", "Health Care",
    "Information Technology", "Internet Services",
    "Lending and Investments", "Manufacturing", "Media and Entertainment",
    "Messaging and Telecommunications", "Mobile", "Music and Audio",
    "Natural Resources", "Navigation and Mapping", "Other", "Payments",
    "Platforms", "Privacy and Security", "Professional Services",
    "Real Estate", "Sales and Marketing", "Science and Engineering",
    "Social Impact", "Software", "Sports", "Sustainability",
    "Transportation", "Travel and Tourism", "Video"
]


cols_to_move = [c for c in to_move if c in companies_seed.columns]

new_order = [c for c in companies_seed.columns if c not in cols_to_move] + cols_to_move

companies_seed = companies_seed[new_order]

Make sure that at least one investor is known for a round

In [ ]:

valid_investor_ids = investors_seed['Investor ID'].unique()
seed_help = seed_help[seed_help['Investor ID'].isin(valid_investor_ids)]

valid_company_ids = seed_help['Company ID'].unique()

initial_company_count = companies_seed['Company ID'].nunique()
print(f"Initial number of companies in companies_seed: {initial_company_count}")

companies_seed = companies_seed[companies_seed['Company ID'].isin(valid_company_ids)]

final_company_count = companies_seed['Company ID'].nunique()
print(f"Final number of companies in companies_seed: {final_company_count}")

dropped_company_count = initial_company_count - final_company_count
print(f"Number of companies dropped: {dropped_company_count}")

In [ ]:
output_path = 'data/sets-for-r/investors_seed.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True) 
investors_seed.to_csv(output_path, index=False)

output_path = 'data/sets-for-r/companies_seed.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)  
companies_seed.to_csv(output_path, index=False)

output_path = 'data/sets-for-r/seed_help.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)  
seed_help.to_csv(output_path, index=False)

output_path = 'data/sets-for-r/rounds.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
rounds_filtered.to_csv(output_path, index=False)